In [ ]:
from models import *
from utils import *

from cfal_loss import *

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.models import Sequential

from keras.callbacks import LearningRateScheduler, Callback,ModelCheckpoint
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score, matthews_corrcoef, cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import average_precision_score

import os
import matplotlib.pyplot as plt
import numpy as np


from keras import backend as K
from tensorflow.keras import layers
from keras.regularizers import l2
from keras.layers import PReLU
from keras import initializers


from keras.layers import Input,Conv2D,Activation,Dense,Lambda,Flatten,Embedding,PReLU,BatchNormalization
from keras.models import Model
import keras.backend as K
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping


# Load KCRC Data (NCT-CRC)

In [ ]:
train_dir = r"C:\data\NCT-CRC-HE-100K/"
test_dir = r"C:\data\CRC-VAL-HE-7K/"

In [ ]:
# Add an extra class for the RW of the CFAL LOSS
classes = ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM','None']

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, validation_split=0.15, preprocessing_function=tf.keras.applications.vgg19.preprocess_input)

train_batches = datagen.flow_from_directory(directory=train_dir, target_size=(112,112), 
                                            classes=classes, batch_size=32,subset='training')
valid_batches= datagen.flow_from_directory(directory=train_dir, target_size=(112,112), 
                                           classes=classes, batch_size=32,subset='validation',shuffle=False)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg19.preprocess_input).flow_from_directory(directory=test_dir, target_size=(112,112), classes=classes, batch_size=32, shuffle=False)

weights_kcrc = [10407, 10566, 11512, 11557, 8896, 13536, 8763, 10446, 14317]


In [ ]:
def calc_effective_weights(counts, beta=0.999):
    effective_num = 1.0 - np.power(beta, counts)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * int(len(counts))
    return weights

eff_weights = calc_effective_weights(weights_kcrc, beta=0.9999)
print(eff_weights)

# Define Model

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa


In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 35
image_size = 112

In [ ]:
num_classes = 10 # one additional
input_shape = (112, 112, 3) 

In [ ]:
def create_cct_model(
    image_size=image_size,
    input_shape=input_shape,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
):

    inputs = layers.Input(input_shape)

    # Encode patches.
    cct_tokenizer = CCTTokenizer(num_conv_layers=conv_layers,positional_emb=positional_emb)
    encoded_patches = cct_tokenizer(inputs)

    # Apply positional embedding.
    if positional_emb:
        pos_embed, seq_length = cct_tokenizer.positional_embedding(image_size)
        positions = tf.range(start=0, limit=seq_length, delta=1)
        position_embeddings = pos_embed(positions)
        encoded_patches += position_embeddings

    # Calculate Stochastic Depth probabilities.
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Create multiple layers of the Transformer block.
    for i in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # Apply sequence pooling.
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # Classify outputs.
  
    logits = ClusteringAffinity(9, 1, 130)(weighted_representation)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [ ]:
cct_model = create_cct_model()

In [ ]:
num_epochs = 1
history = run_experiment(cct_model, train_batches, valid_batches, batch_size , num_epochs)

# Load Best Model and Test

In [ ]:
filename = ''
cct_model.load_weights(filename)

In [ ]:
Y_pred = cct_model.predict(test_batches)
y_pred = np.argmax(Y_pred ,axis =1)
y_true = test_batches.classes
print(classification_report(y_true, y_pred,digits=4))